<a href="https://colab.research.google.com/github/kospi-2025/EVT/blob/main/MRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
base_url = "https://raw.githubusercontent.com/kospi-2025/EVT/main/"

df = pd.read_csv(base_url + "source_data/" + "ticker_info.csv")
df['id'] = df['id'].astype(str).str.zfill(6)
df['Yahoo_Ticker'] = df['id'] + ".KS"

ticker_to_name = dict(zip(df["Yahoo_Ticker"], df["name"]))
ticker_to_sector = dict(zip(df["Yahoo_Ticker"], df["sector"]))

In [3]:
logDD = pd.read_csv(base_url + "logDD.csv", index_col=0)

# MRL

In [4]:
results = []

qs = np.linspace(0.01, 0.99, 99)  # or reduce for speed

for ticker in logDD.columns:
    series = logDD[ticker].dropna().sort_index()
    us = series.quantile(qs)

    for q, u in zip(qs, us):
        excesses = series[series > u] - u
        if len(excesses) >= 1:  # MRL can be computed if at least one exceedance
            mrl = excesses.mean()
            results.append({
                "Ticker": ticker,
                "Quantile": q,
                "Threshold": u,
                "MRL": mrl,
                "NumExceedances": len(excesses)
            })

# Save results
temp = pd.DataFrame(results)
temp.to_csv("MRL.csv", index=False)

In [5]:
temp

,Ticker,Quantile,Threshold,MRL,NumExceedances
0,000080.KS,0.01,2.883931e-08,0.017128,3847
1,000080.KS,0.02,8.296211e-08,0.017303,3808
2,000080.KS,0.03,1.416458e-07,0.017482,3769
3,000080.KS,0.04,1.402012e-03,0.016257,3730
4,000080.KS,0.05,1.735412e-03,0.016094,3691
...,...,...,...,...,...
19003,402340.KS,0.95,6.085221e-02,0.023841,31
19004,402340.KS,0.96,6.350512e-02,0.026634,25
19005,402340.KS,0.97,7.097869e-02,0.026989,19
19006,402340.KS,0.98,9.131972e-02,0.014504,13


In [6]:
19008/99

192.0